In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

Requirements
  1. numpy
  2. multiprocessing


## <span style="color:#44FF44;">Using multiprocessing.pool</span>
1. pool.map

In [ ]:
! python 00_tuto.py 1
! python 00_tuto.py 0


2. Job numbers and reducing execution time

In [ ]:
# ! python 01_tuto.py 1
! python 01_tuto.py 10
! python 01_tuto.py 14
! python 01_tuto.py 16
! python 01_tuto.py 18
! python 01_tuto.py 20
! python 01_tuto.py 25

3. Using `pool.starmap`
   
For more than 1 argument

In [ ]:
! python 02_tuto.py 10

4. Using `pool.apply_async`

More control over the multiprocessing tasks <br>
Assigning multiple functions in a session <br>
Releassing the parent process for other tasksto do <br>

In [ ]:
! python 03_tuto.py

## <span style="color:#FFAA00;">Using multiprocessing.Process</span>
Let's leave this for children to play with and focus more on cool stuff<br>
Also, multiprocessing.Process is friendly with notebooks but... 


In [ ]:
import multiprocessing

# Define a function to be executed by the process
def worker_function(name):
    print(f"Hello, {name}!")

if __name__=='__main__':
  # Create a multiprocessing.Process instance
  process1 = multiprocessing.Process(target=worker_function, args=("Charlie the Harley"  ,))
  process2 = multiprocessing.Process(target=worker_function, args=("Sophie the trophy"   ,))
  process3 = multiprocessing.Process(target=worker_function, args=("Albert the shepherd" ,))
  
  # Start the process
  process1.start()
  process2.start()
  process3.start()

  # Wait for the process to finish (optional)
  print("Main program continues...")

  process1.join()
  process2.join()
  process3.join()

In [ ]:
! python 04_tuto.py

In the notebook: 
1. The kernel will kill itself (suicide) if any cell ended running and a process is alive. join all processes before ending a cell
2. If `Process` is called directly inside the notebook, the children can't reach the casting interface (can't `print()`) but it's possible if the Process is inside a module/script that have been imported by the notebook.
3. In any python script, `Process` and `Pool` should be called in main scope (`if __name__=='__main__':...`) because the script is called inside all processes.
4. We can use args or kwargs arguments in `Process` to pass arguments or keyworded arguments.
5. There is no number of processes to specify in this case so it should be manually done 

In [ ]:
! python 05_tuto.py

`for ind,p in enumerate(processes): `<br>  
&emsp;`if not p.is_alive():`<br>
&emsp;&emsp;`p.close()`<br>
&emsp;&emsp;`processes.pop(p)`<br>

It's to close all the processes that have ended their jobs. This is important if you have a lot of processes >100s. There is some files created and opened and all systems have a limited amount of memory for opening files simultaneously.

## <span style="color:#FF5500;">Sharing  memory</span>

In [ ]:
from multiprocessing import shared_memory
import numpy as np
# Creating a parent matrix
matrix = np.random.randint(0,10,size=(3,))
matrix.nbytes

shm_a = shared_memory.SharedMemory(create=True, size=matrix.nbytes)
shm_matrix= np.ndarray(shape=matrix.shape,dtype=matrix.dtype, buffer=shm_a.buf)
shm_matrix[:] =matrix

It's important to fill the shared matrix using slices (`[:]`) else equalizing two matrices directly will break the link and delete the buffer and build a new variable

In [ ]:
shm_matrix2 = np.ndarray(shape=matrix.shape,dtype=matrix.dtype, buffer=shm_a.buf)
shm_matrix3 = np.ndarray(shape=matrix.shape,dtype=matrix.dtype, buffer=shm_a.buf)
print(shm_matrix2,shm_matrix3)

In [ ]:
shm_matrix3[0] = 100
print(shm_matrix2,shm_matrix3)

In [ ]:
!python 07_tuto.py 1 3

### The small cherry on the top

In [ ]:
from multiprocessing.shared_memory import SharedMemory
from time import sleep
shape =(3,) 
dtype = int 
shmm_name = "wnsm_0065ebd3"

shmm = SharedMemory(create=False,name=shmm_name)
data = np.ndarray(shape,dtype,shmm.buf)
while True:
  print(data)
  sleep(0.1)
